In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")


Using device: cuda


In [ ]:
!pip install fiftyone

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.0/943.0 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 14.4 MB/s eta 0:0

#**LOAD DATASET**#

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import random
from collections import defaultdict
from fiftyone import ViewField as F

In [ ]:
# Xử lý tập train
dataset_train = foz.load_zoo_dataset("coco-2017", split="train")

# Xử lý tập val
dataset_val = foz.load_zoo_dataset("coco-2017", split="validation")


INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/coco-2017/train' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


Images already downloaded


INFO:fiftyone.utils.coco:Images already downloaded


...

INFO:fiftyone.zoo.datasets:Loading 'coco-2017' split 'validation'


 100% |███████████████| 5000/5000 [29.5s elapsed, 0s remaining, 152.4 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 5000/5000 [29.5s elapsed, 0s remaining, 152.4 samples/s]      


Dataset 'coco-2017-validation' created


INFO:fiftyone.zoo.datasets:Dataset 'coco-2017-validation' created


In [ ]:
relevant_classes = [
    "person", "bicycle", "car", "motorcycle", "bus",
    "train", "truck", "traffic light"
]

In [ ]:
def filter_and_balance_dataset(dataset, split_name, max_samples_per_class=1500):
    """
    Lọc dataset chỉ giữ lại ảnh chứa các class quan trọng,
    sau đó lấy tối đa 1500 ảnh cho mỗi class.
    """
    # Lọc dataset chỉ giữ lại các ảnh có object thuộc nhóm này
    dataset_filtered = dataset.filter_labels("ground_truth", F("label").is_in(relevant_classes))

    # Nhóm ảnh theo từng class
    class_samples = {cls: [] for cls in relevant_classes}
    for sample in dataset_filtered:
        labels = [det.label for det in sample.ground_truth.detections]
        for label in labels:
            if label in class_samples:
                class_samples[label].append(sample)

    # Chọn ngẫu nhiên tối đa 1500 ảnh cho mỗi class
    selected_samples = []
    for cls, samples in class_samples.items():
        selected_samples.extend(random.sample(samples, min(max_samples_per_class, len(samples))))

    # Xuất dữ liệu ra thư mục riêng cho từng tập
    dataset_final = fo.Dataset(name=f"coco_av_{split_name}")
    dataset_final.add_samples(list(selected_samples))
    dataset_final.export(
        export_dir=f"/content/coco_av/{split_name}",
        dataset_type=fo.types.YOLOv5Dataset,
        label_field="ground_truth",
        split=split_name
    )

    # Đếm số lượng ảnh chứa từng class
    class_counts = defaultdict(int)
    for sample in selected_samples:
        labels = set([det.label for det in sample.ground_truth.detections])
        for label in labels:
            class_counts[label] += 1

    # Hiển thị kết quả đếm ảnh của từng class
    print(f"✅ Xuất tập {split_name} hoàn tất! Tổng số ảnh: {len(selected_samples)}")
    print(f"📊 Thống kê số ảnh chứa mỗi class trong {split_name}:")
    for cls, count in class_counts.items():
        print(f"   - {cls}: {count} ảnh")


In [ ]:
filter_and_balance_dataset(dataset_train, "train")


 100% |█████████████| 12000/12000 [2.0m elapsed, 0s remaining, 95.5 samples/s]       


INFO:eta.core.utils: 100% |█████████████| 12000/12000 [2.0m elapsed, 0s remaining, 95.5 samples/s]       


 100% |█████████████| 12000/12000 [1.5m elapsed, 0s remaining, 163.4 samples/s]      


INFO:eta.core.utils: 100% |█████████████| 12000/12000 [1.5m elapsed, 0s remaining, 163.4 samples/s]      


✅ Xuất tập train hoàn tất! Tổng số ảnh: 12000
📊 Thống kê số ảnh chứa mỗi class trong train:
   - person: 8789 ảnh
   - car: 5961 ảnh
   - train: 1855 ảnh
   - bus: 2846 ảnh
   - truck: 3638 ảnh
   - traffic light: 3199 ảnh
   - bicycle: 2399 ảnh
   - motorcycle: 2367 ảnh


In [ ]:
!ls /content/coco_av/train/labels/train | wc -l
!ls /content/coco_av/val/labels/val | wc -l

9071
1530


In [ ]:

val_filtered = dataset_val.filter_labels("ground_truth", F("label").is_in(relevant_classes))
subset = val_filtered.take(int(len(val_filtered) * 0.5))


# Xuất dữ liệu theo format YOLOv5
subset.export(
    export_dir="/content/coco_av/val",
    dataset_type=fo.types.YOLOv5Dataset,
    label_field="ground_truth",
    split="val"
)
print("✅ Xuất dữ liệu hoàn tất!")

 100% |███████████████| 1530/1530 [4.8s elapsed, 0s remaining, 270.1 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 1530/1530 [4.8s elapsed, 0s remaining, 270.1 samples/s]      


✅ Xuất dữ liệu hoàn tất!


In [ ]:
from collections import Counter

# Đếm số ảnh chứa từng class
class_counts = Counter()
for sample in subset:
    labels = [det.label for det in sample.ground_truth.detections]
    unique_labels = set(labels)  # Chỉ tính mỗi ảnh 1 lần cho mỗi class
    class_counts.update(unique_labels)

# In kết quả
for cls, count in class_counts.items():
    print(f"{cls}: {count} ảnh")


person: 1354 ảnh
car: 253 ảnh
train: 81 ảnh
truck: 119 ảnh
motorcycle: 79 ảnh
traffic light: 91 ảnh
bus: 86 ảnh
bicycle: 75 ảnh


In [ ]:
import shutil
import os

# Đường dẫn file dataset.yaml trong thư mục train
train_yaml_path = "/content/coco_av/train/dataset.yaml"
# Đường dẫn file dataset.yaml mong muốn
target_yaml_path = "/content/coco_av/dataset.yaml"
# Đường dẫn file dataset.yaml trong thư mục val
val_yaml_path = "/content/coco_av/val/dataset.yaml"

# Di chuyển file dataset.yaml từ train ra /content/coco_av
shutil.move(train_yaml_path, target_yaml_path)

# Xóa file dataset.yaml trong thư mục val nếu tồn tại
if os.path.exists(val_yaml_path):
    os.remove(val_yaml_path)
    print("✅ Đã xóa /content/coco_av/val/dataset.yaml")
else:
    print("⚠️ Không tìm thấy /content/coco_av/val/dataset.yaml")

print("✅ Đã di chuyển dataset.yaml từ train ra /content/coco_av")


✅ Đã xóa /content/coco_av/val/dataset.yaml
✅ Đã di chuyển dataset.yaml từ train ra /content/coco_av


In [ ]:
!cat /content/coco_av/dataset.yaml

names:
  0: car
  1: person
  2: train
  3: bus
  4: truck
  5: traffic light
  6: bicycle
  7: motorcycle
path: /content/coco_av/
train: ./train/images/train/
val: ./val/images/val/

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 977.1/977.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 818.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

#**TRAIN**#

In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model = YOLO("yolo11n.pt")

In [ ]:
model.train(data = "/content/coco_av/dataset.yaml", epochs = 20, imgsz = 640, batch = 8, workers = 2)

Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/coco_av/dataset.yaml, epochs=20, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning /content/coco_av/train/labels/train.cache... 9071 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9071/9071 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/coco_av/train/images/train/000000099844.jpg: 2 duplicate labels removed


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/coco_av/val/labels/val.cache... 1530 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1530/1530 [00:00<?, ?it/s]


...

      20/20      5.94G        1.3      1.078      1.178         53        640: 100%|██████████| 1134/1134 [03:40<00:00,  5.14it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 96/96 [00:14<00:00,  6.60it/s]


                   all       1530       7378      0.427      0.348      0.381      0.256

20 epochs completed in 1.357 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.4MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 96/96 [00:17<00:00,  5.54it/s]


                   all       1530       7378      0.425      0.349      0.381      0.256
                   car        253        836      0.677      0.547      0.591      0.378
                person       1354       5477       0.77      0.612      0.708      0.458
                 train         81         97      0.713      0.835      0.853      0.687
                   bus         79        199          0          0   0.000427   0.000171
                 truck        119        200      0.533       0.35      0.404      0.262
         traffic light         91        309      0.682      0.414      0.487      0.259
               bicycle         86        124          0          0   9.01e-05   6.04e-05
            motorcycle         75        136     0.0265     0.0294    0.00805    0.00376
Speed: 0.3ms preprocess, 2.6ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a38d071d210>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,

In [ ]:
metrics = model.val(data="/content/coco_av/dataset.yaml", split="train")
print(metrics)


Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


val: Scanning /content/coco_av/train/labels/train.cache... 9071 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9071/9071 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/coco_av/train/images/train/000000099844.jpg: 2 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 567/567 [01:47<00:00,  5.28it/s]


                   all       9071      89388       0.73      0.574      0.652      0.437
                   car       4104      18935      0.711      0.541      0.617      0.393
                person       6570      42233      0.767      0.554      0.661      0.409
                 train       1540       2220      0.846      0.795      0.866      0.667
                   bus       1879       3421       0.78      0.733      0.808      0.642
                 truck       2473       4807       0.64      0.487      0.563        0.4
         traffic light       2043       7746      0.717      0.436      0.515      0.281
               bicycle       1521       4561      0.634      0.466      0.522      0.298
            motorcycle       1510       5465      0.746      0.577      0.669      0.409
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array(

#**SAVE MODEL**#

In [ ]:
from google.colab import files

# Đường dẫn đến mô hình tốt nhất sau khi train
model_path = "/content/runs/detect/train/weights/best.pt"

# Tải mô hình về máy
files.download(model_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>